# Importar librerias

In [1]:
import pandas as pd
from modules import AthenaQueryExecutor, SqlRender
from datetime import datetime
import datetime
import re
import os
import shutil

from extra.config import awsconfig
from extra.utils import (extract, load, utils)
from extra.constants import (trx_columns, data_id_pago, name_columns_masivo)
from extra import transforms 

# AWS Profile

In [2]:
awsconfig.aws_config_profile('NEQUI-DataBusinessAnalyst-Role-044879804046')
# # Establecer el perfil que deseas usar
# profile_name = 'NEQUI-DataBusinessAnalyst-Role-044879804046'
# # Configurar variable de entorno para AWS_PROFILE
# os.environ['AWS_PROFILE'] = profile_name

Perfil AWS configurado: NEQUI-DataBusinessAnalyst-Role-044879804046


## Metodo para crear archivo historico del masivo

In [3]:
def creacion_archivo_historico(df_hist):
    df_hist['Fecha creación reclamo'] = pd.to_datetime(df_hist['created_at']).apply(lambda x: x.date().strftime('%d/%m/%y'))
    df_hist['Fecha escalada a Opercaiones'] = None
    df_hist['Rad. Zendesk'] = df_hist['ticket_id']
    df_hist['Núm. documento afectad@'] = df_hist['numero_de_documento']
    df_hist['Núm. cuenta suplantada'] = df_hist['numero_de_cuenta_870']
    df_hist['Nombre completo'] = df_hist['nombre_del_solicitante']
    df_hist['Producto'] = df_hist['numero_de_cuenta_870'].apply(lambda x: 'Cuenta' if(str(x)[:3]=='870') else 'credito')
    df_hist['Transacción Datacrédito'] = None
    df_hist['Rad. Zendesk de seguimiento'] = None
    df_hist['Fecha gestión'] = datetime.datetime.now().date().strftime('%d/%m/%y')
    df_hist['Equipo solución'] = 'Operaciones'
    df_hist['CIERRE APR'] = df_hist['Error Description']
    df_hist['Fecha respuesta reclamo'] = None
    df_hist['concat 1'] = df_hist['Rad. Zendesk'].apply(lambda x: f"Suplantación de identidad no reconocida en ticket {x}")
    df_hist['concat 2'] = df_hist.apply(lambda x: f"Se baja reporte en Datacredito para usuario con CC {x['numero_de_documento']}", axis=1)
    cols = ['Fecha creación reclamo', 'Fecha escalada a Opercaiones', 'Rad. Zendesk', 
            'Núm. documento afectad@', 'Núm. cuenta suplantada', 'Nombre completo',
            'Producto', 'Transacción Datacrédito', 'Rad. Zendesk de seguimiento', 
            'Fecha gestión', 'Equipo solución', 'CIERRE APR', 'Fecha respuesta reclamo',
            'concat 1', 'concat 2']
    return df_hist[cols]


## Lectura del APR (automatico en folder input)

In [4]:
# 1. Listar archivos .xlsx que hay en el folder input/ y leer el df (Solo 1 archivo)

# Obtener el directorio padre
dir_project = os.path.dirname(os.getcwd())

# Directorio input data de insumo
dir_data = os.path.join(dir_project, 'co_gestionFraude_eliminacionMasivaExperian/data/input')
# Patron regex para validar nombre del archivo
regex_partition = r'^APRCIERRE[^\d]*(\d{8})(\d{2}).*.xlsx$'

# Lista para almacenar los nombres de archivos Excel en el directorio
archivos_excel = []

# Iterar sobre los archivos en el directorio
for archivo in os.listdir(dir_data):
    if archivo.endswith('.xlsx'):  # Puedes ajustar esto según el formato de tus archivos Excel
        archivos_excel.append(archivo)
        print(archivos_excel)

# Verificar la cantidad de archivos Excel
cantidad_archivos = len(archivos_excel)

if cantidad_archivos == 0:
    print("No se encontraron archivos Excel en el directorio.")
elif cantidad_archivos == 1:

    print("Archivo unico")
    #   Leer el archivo Excel
        # Validar el nombre del archivo con la expresión regular
    if re.match(regex_partition, archivos_excel[0]):
            file_input_path = os.path.join(dir_data, archivos_excel[0])
            print("File nombrado OK")
            df_raw = pd.read_excel(file_input_path)
    else:
         print("Archivo mal nombrado")
else:
     print("Hay más de un archivo")


df = df_raw.copy()
print(df.columns)

print(df.info())
df.columns = [i.strip() for i in df.columns]
for i in df.columns:
    if i not in ("Upload Date", "Account Number", "Reason Code", "Tran Date", "Transfer Account Number"):
        print(i)
        df[i] = df[i].apply(lambda x: x.strip())

print(f"\n*****\nRegistros del archivo de entrada APR enviado por Operaciones: {df.shape}")

df_copy = df.copy()

['APRCIERREMLZ2025051300.xlsx']
Archivo unico
File nombrado OK
Index(['File Name                     ', 'Upload Date ', 'Status    ',
       'Error Description                                 ',
       'Account Number  ', 'Transfer Account Number', 'Reason Code  ',
       'Closure Notes                                     ', 'Tran Id   ',
       'Tran Date   ', 'User Part Tran Code  ',
       'Tran particulars                                  '],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 12 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   File Name                                           65 non-null     object        
 1   Upload Date                                         65 non-null     datetime64[ns]
 2   Status                                              65 non-null    

In [5]:
# df_columns_str = df.select_dtypes(include=['object', 'string'])

# # Obtener los nombres de las columnas seleccionadas
# name_columns_str = df_columns_str.columns.tolist()

# name_columns_str.remove('Tran Date')

# name_columns_str

# for i in name_columns_str:
#     print(i)
#     df[i] = df[i].apply(lambda x: x.strip())



## Filtro de los casos para bajar el reporte con el código de Experian

In [6]:
# Se filtran sólo las que se les bajará el reporte por parte de operaciones
# Account Closed Successfully y Account is Already Closed son parametros validos para seguir
df = df[(df['Closure Notes'] == 'Suplantacion')&((df['Error Description']=='Account Closed Successfully')|(df['Error Description']=='Account is Already Closed'))].reset_index(drop=True)
accounts = tuple([f'{i}' for i in df['Account Number']])
accounts_list = list(accounts)
accounts_tuple = str(accounts)
accounts_list = tuple(accounts_list)

print(f"\n*****\nCuentas que cumplen condiciones a cerrar: {len(accounts_list)}")


*****
Cuentas que cumplen condiciones a cerrar: 65


## Lectura datos desde Glue Catalog

In [7]:
# Se envian parametros para armar el query con ayuda de la plantilla.
query_string_zendesk = SqlRender.render_template(name_template="consulta.sql", 
                                                 database="co_delfos_servicio_raw_pdn_rl",
                                                 table="co_zendesk_flatten_tickets", 
                                                 accounts_list = accounts_list)
print(query_string_zendesk)
#Se le envia el query a Athena para que retorne la información en un dataframe.
df_athena_zendesk = AthenaQueryExecutor.execute_query(query=query_string_zendesk,
                                                      database_athena="co_delfos_servicio_raw_pdn_rl",
                                                      workgroup_athena="delfos")

SELECT  
    ticket_id,
    created_at,
    fields_payload['numero_de_cuenta_870'] as numero_de_cuenta_870,
    fields_payload['nombre_del_solicitante'] as nombre_del_solicitante,
    fields_payload['tipo_de_documento'] as tipo_documento,
    fields_payload['numero_de_documento'] as numero_de_documento,
    fields_payload['celular_de_quien_se_comunica'] as celular_de_quien_se_comunica,
    CONCAT(
        RPAD(fields_payload['numero_de_cuenta_870'], 18, '0'),
        LPAD(fields_payload['numero_de_documento'], 11, '0'), 
        (CASE 
            WHEN fields_payload['tipo_de_documento'] = 'tarjeta_cédula_de_extranjería' THEN '4' 
            WHEN fields_payload['tipo_de_documento'] = 'tarjeta_tarjeta_de_identidad' THEN '7'
            ELSE '1' END),
        'E') as code_experian -- genera el codigo experian con la longitud necesaria para crear el masivo por registro
        -- E significa que se baja el reporte por que "Se presento Fraude"
FROM co_delfos_servicio_raw_pdn_rl.co_zendesk

In [8]:
print(f"Registros encontrados en Glue Catalog: {df_athena_zendesk.shape}")

Registros encontrados en Glue Catalog: (75, 8)


## Limpieza de los datos

In [9]:
# Se eliminan duplicados, dejando solo ultimo ticket creado con diferente 870 (no duplicado)
df_final = df_athena_zendesk.sort_values('created_at')\
                            .drop_duplicates(subset=['numero_de_cuenta_870'], keep='last')\
                            .reset_index(drop=True)

# Exportar archivos de salida segun la particion de cada APR

### Funciones para exportar archivos

In [10]:
def verificar_y_crear(nombre_archivo, carpeta, dataframe):
    # Verificar si el archivo ya existe en la carpeta
    if os.path.exists(os.path.join(carpeta, nombre_archivo)):
        # Si existe, agregar "_2" al nombre del archivo
        nombre_sin_extension, extension = os.path.splitext(nombre_archivo)
        nuevo_nombre = f"{nombre_sin_extension}_2{extension}"
        # Verificar si el nuevo nombre ya existe
        while os.path.exists(os.path.join(carpeta, nuevo_nombre)):
            # Si existe, incrementar el número al final del nombre
            indice = nuevo_nombre.rfind("_")
            numero = int(nuevo_nombre[indice + 1: -len(extension)])
            nuevo_nombre = f"{nombre_sin_extension}_{numero + 1}{extension}"
        # Crear el nuevo archivo con el nuevo nombre
        if extension == ".txt":
            dataframe.to_csv(os.path.join(carpeta, nuevo_nombre), index=False, header=False ,sep='\t')
        else:
            dataframe.to_csv(os.path.join(carpeta, nuevo_nombre), index=False, sep='\t')
        print(f"El archivo {nombre_archivo} ya existe. Se creó {nuevo_nombre}.")
        # Condicion para crear o no el archivo xlsm, ya que no todos deben quedar con .xlsx
        if coinciden_iniciales(nombre_archivo) == False:
            guardar_como_excel(os.path.join(carpeta, nuevo_nombre))
        
    else:
        nombre_sin_extension, extension = os.path.splitext(nombre_archivo)
        # Si no existe, crear el archivo con el nombre original
        if extension == ".txt":
            dataframe.to_csv(os.path.join(carpeta, nombre_archivo), index=False, header=False, sep='\t')
        else:
            dataframe.to_csv(os.path.join(carpeta, nombre_archivo), index=False, sep='\t')
        print(f"Se creó el archivo {nombre_archivo}.")
        # Condicion para crear o no el archivo xlsm, ya que no todos deben quedar con .xlsx
        if coinciden_iniciales(nombre_archivo) == False:
            guardar_como_excel(os.path.join(carpeta, nombre_archivo))
        
def guardar_como_excel(ruta_archivo_csv):
    # Ruta del archivo CSV o TXT

    # Determina si es un archivo CSV o TXT
    if ruta_archivo_csv.lower().endswith('.csv'):
        # Lee el archivo CSV en un DataFrame
        df = pd.read_csv(ruta_archivo_csv, delimiter='\t')
    elif ruta_archivo_csv.lower().endswith('.txt'):
        # Lee el archivo TXT en un DataFrame, utilizando tabulaciones como delimitador
        df = pd.read_csv(ruta_archivo_csv, delimiter='\t')
    else:
        print("El archivo no es compatible. Debe ser un archivo CSV o TXT.")
        exit()

    # Genera el nombre del archivo Excel
    archivo_destino = ruta_archivo_csv[:-4] + '.xlsx'

    # Guarda el DataFrame como un archivo Excel
    df.to_excel(archivo_destino, index=False)

    # print(f'Se ha creado el archivo "{archivo_destino}" en formato XLSX.')

def coinciden_iniciales(name):
    lista = ["Bloqueo_2", "df_final", "APRCierre"]
    for elemento in lista:
        if name.startswith(elemento):
            return True
    return False

def eliminar_APR(ruta_archivo):
    if os.path.exists(ruta_archivo):
        # Eliminar el archivo
        os.remove(ruta_archivo)
        print(f"El archivo en la ruta {ruta_archivo} ha sido eliminado exitosamente.")
    else:
        print(f"No se encontró ningún archivo en la ruta {ruta_archivo}.")

def copiar_y_pegar_archivo(origen, destino):
    # Comprobar si el archivo ya existe en la carpeta de destino
    if os.path.exists(destino):
        nombre_archivo = os.path.basename(origen)
        # Obtener el nombre del archivo sin extensión
        nombre_base, extension = os.path.splitext(nombre_archivo)
        # Buscar un nombre único para el nuevo archivo
        indice = 1
        while os.path.exists(os.path.join(destino, nombre_archivo)):
            nombre_archivo = f"{nombre_base}_{indice}{extension}"
            indice += 1
        destino = os.path.join(destino, nombre_archivo)
    # Copiar y pegar el archivo
    shutil.copy2(origen, destino)

## 1. Exportar file APR.txt
- Tener en cuenta exportar con el nombre de "masivo_APR_<fecha_APR_Original>.txt"

In [11]:
"""
Extraer particion del nombre del archivo original (insumo)
"""

# Aplicar la expresión regular al nombre ORIGINAL del archivo
match = re.search(regex_partition, archivos_excel[0])

# Verificar si se encontró el grupo de captura (particion)
if match:
    partition = match.group(1)
else:
    print("No se encontró el número en el nombre del archivo.")

"""
Almacenar en las particiones del folder data/output
"""

# Directorio data output
dir_data_output = os.path.join(dir_project, 'co_gestionFraude_eliminacionMasivaExperian/data/output')

# Concatenar ruta destino con particion
folder_destino = os.path.join(dir_data_output, f'{partition[0:4]}/{partition[4:6]}/{partition[6:8]}')

# Si el folder no existe, crearlo
if not os.path.exists(folder_destino):
    os.makedirs(folder_destino)

# Archivo original (name + dir)
name_original_file_input = f'APRCierre_{partition}.csv'
dir_original_file_input = os.path.join(folder_destino, name_original_file_input)

# Archivo procesado txt masivo experian (name + dir)
name_txt_file_output = f'Bloqueo_{partition}.txt'
dir_xlsx_file_output = os.path.join(folder_destino, name_txt_file_output)

# Archivo csv masivo experian procesado (name + dir)
name_csv_file_output = f'Bloqueo_{partition}.csv'
dir_csv_file_output = os.path.join(folder_destino, name_csv_file_output)

# Archivo csv masivo experian procesado (name + dir)
name_df_final_csv_file_output = f'df_final_{partition}.csv'
dir_df_final_csv_file_output = os.path.join(folder_destino, name_csv_file_output)

# Mover el archivo original

copiar_y_pegar_archivo(file_input_path, folder_destino)
# shutil.copy(file_input_path, dir_original_file_input)

# # Creando archivo APR original
verificar_y_crear(name_original_file_input, folder_destino, df_copy)
# Creando archivo de salida masivo .txt
verificar_y_crear(name_txt_file_output, folder_destino, df_final.code_experian)
# Creando archivo de salida masivo .csv
verificar_y_crear(name_csv_file_output, folder_destino, df_final.code_experian)
# Creando archivo df_final
verificar_y_crear(name_df_final_csv_file_output, folder_destino, df_final)

Se creó el archivo APRCierre_20250513.csv.
Se creó el archivo Bloqueo_20250513.txt.
Se creó el archivo Bloqueo_20250513.csv.
Se creó el archivo df_final_20250513.csv.


## 2. Exportar el file de historico en formato .xlsx
- Tener en cuenta exportar con el nombre de "masivo_APR_<fecha_APR_Original>.xlsx"

In [12]:
import numpy as np
"""
Limpieza datos
"""
# Se muestran los Errores de tipificación en 870 comparando el 870 del APR con el del ticket
df_final['numero_de_cuenta_870'] = df_final['numero_de_cuenta_870'].astype(np.int64) # Se comenta esta linea porque esta generando un error el tipado de la columna y no permite cruzar nada en el join.
error_accounts_list = []
for i in accounts_list:
    if i not in list(df_final['numero_de_cuenta_870'].astype(str)):
        error_accounts_list.append(i)
print(f"No se encontraron tickets para los siguientes números de cuenta: {error_accounts_list}")

# Se genera archivo para llevar registro histórico
df_hist = df.merge(df_final, how='inner', 
                   left_on='Account Number', 
                   right_on='numero_de_cuenta_870')

reporte = creacion_archivo_historico(df_hist)

"""
Exportar archivo .xlsx
"""
name_xlsx_file_output = f'Bloqueo_hist_{partition}.csv'
verificar_y_crear(name_xlsx_file_output, folder_destino, reporte)

No se encontraron tickets para los siguientes números de cuenta: ['87077511281', '87077512530', '87075165629']
Se creó el archivo Bloqueo_hist_20250513.csv.


In [13]:
# Eliminando APR de la ruta input
eliminar_APR(file_input_path)

El archivo en la ruta c:\Users\disanchez\OneDrive - PersonalSoft S.A.S\Analítica\Github\co_gestionFraude_eliminacionMasivaExperian/data/input\APRCIERREMLZ2025051300.xlsx ha sido eliminado exitosamente.
